In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [32]:
df = pd.read_csv('train.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df.dropna(inplace= True)

In [34]:
visa = pd.get_dummies(df['VISA_CLASS'], drop_first=True)
df = pd.concat([df,visa],axis=1)
df.drop('VISA_CLASS', axis=1,inplace=True)

In [35]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['FULL_TIME_POSITION'] = labelencoder.fit_transform(df['FULL_TIME_POSITION'].astype(str))
df['CASE_SUBMITTED_YEAR'] = labelencoder.fit_transform(df['CASE_SUBMITTED_YEAR'].astype(int))
df['DECISION_YEAR'] = labelencoder.fit_transform(df['DECISION_YEAR'].astype(int))
df['EMPLOYER_STATE'] = labelencoder.fit_transform(df['EMPLOYER_STATE'].astype(str))
df['SOC_NAME'] = labelencoder.fit_transform(df['SOC_NAME'].astype(str))
df['NAICS_CODE'] = labelencoder.fit_transform(df['NAICS_CODE'].astype(int))
df['WILLFUL_VIOLATOR'] = labelencoder.fit_transform(df['WILLFUL_VIOLATOR'].astype(str))
df['H-1B_DEPENDENT'] = labelencoder.fit_transform(df['H-1B_DEPENDENT'].astype(str))
df['WORKSITE_STATE'] = labelencoder.fit_transform(df['WORKSITE_STATE'].astype(str))
df['WORKSITE_POSTAL_CODE'] = labelencoder.fit_transform(df['WORKSITE_POSTAL_CODE'].astype(str))
df['WAGE_UNIT_OF_PAY'] = labelencoder.fit_transform(df['WAGE_UNIT_OF_PAY'].astype(str))
df['PW_SOURCE_YEAR'] = labelencoder.fit_transform(df['PW_SOURCE_YEAR'].astype(int))
df['CASE_STATUS'] = labelencoder.fit_transform(df['CASE_STATUS'].astype(str))
df['PW_SOURCE'] = labelencoder.fit_transform(df['PW_SOURCE'].astype(str))
df['PW_SOURCE_OTHER'] = labelencoder.fit_transform(df['PW_SOURCE_OTHER'].astype(str))
df['PW_UNIT_OF_PAY'] = labelencoder.fit_transform(df['PW_UNIT_OF_PAY'].astype(str))

In [36]:
minority_class_len = len(df[df['CASE_STATUS'] == 2])


In [37]:
majority_class0_indices = df[df['CASE_STATUS'] == 0].index
majority_class1_indices = df[df['CASE_STATUS'] == 1].index
majority_class3_indices = df[df['CASE_STATUS'] == 3].index
print(majority_class0_indices)

Int64Index([ 23966,  23971,  23972,  23983,  23989,  23995,  23998,  24000,
             24007,  24010,
            ...
            418056, 418057, 418058, 418059, 418060, 418061, 418063, 418064,
            418065, 418066],
           dtype='int64', length=361759)


In [38]:
random_majority_class0_indices = np.random.choice(majority_class0_indices,
                                                 minority_class_len,
                                                replace=False)
random_majority_class1_indices = np.random.choice(majority_class1_indices,
                                                 minority_class_len,
                                                 replace=False)
random_majority_class3_indices = np.random.choice(majority_class3_indices,
                                                 minority_class_len,
                                                 replace=False)

In [39]:
minority_class_indices = df[df['CASE_STATUS'] == 2].index

In [40]:
undersample_indices = np.concatenate([minority_class_indices, random_majority_class0_indices, random_majority_class1_indices,random_majority_class3_indices])

In [41]:
undersample = df.loc[undersample_indices]

In [42]:
from sklearn.model_selection import train_test_split
X = undersample.drop(['CASE_NO', 'CASE_STATUS', 'EMPLOYER_NAME', 'EMPLOYER_COUNTRY'],axis=1)
Y = undersample['CASE_STATUS']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)

In [ ]:
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
clf = XGBClassifier()
selector = RFECV(estimator=clf)
selector.fit(X_train,y_train)

/usr/local/lib64/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [29]:
predictions = selector.predict(X_test.values)

In [30]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.97      0.61      0.75    108555
           1       0.30      0.80      0.44      9975
           2       0.03      0.97      0.06        38
           3       0.09      0.63      0.16      3782

   micro avg       0.63      0.63      0.63    122350
   macro avg       0.35      0.75      0.35    122350
weighted avg       0.89      0.63      0.71    122350



In [27]:
from sklearn.model_selection import train_test_split
X = df.drop(['CASE_NO', 'CASE_STATUS', 'EMPLOYER_NAME', 'EMPLOYER_COUNTRY'],axis=1)
Y = df['CASE_STATUS']